In [1]:
import pandas as pd
import numpy as np
from functools import partial
from ctcv_helper import print_table
from ctcv_helper import read_dataframe
from ctcv_helper import read_dataframe_ratio

In [19]:
# Read limits from csv files
df_fits = pd.DataFrame.from_csv("fits_cv_0p5.csv", sep=",", index_col=None)
df_fits = df_fits.append(pd.DataFrame.from_csv("fits_cv_1p0.csv", sep=",", index_col=None), ignore_index=True)
df_fits = df_fits.append(pd.DataFrame.from_csv("fits_cv_1p5.csv", sep=",", index_col=None), ignore_index=True)

In [20]:
# Read the cross sections from csv files
df_xsbr = {}
df_br = {}
df_xsecs = {}
for model in ['K4', 'K5', 'K6']:
    df_xsbr[model]  = pd.DataFrame.from_csv("xsbr_scalings_%s.csv"%model, sep=",", index_col=None)
    df_br[model]    = pd.DataFrame.from_csv("br_scalings_%s.csv"%model,   sep=",", index_col=None)
    df_xsecs[model] = pd.DataFrame.from_csv("xsecs_tH_ttH_%s.csv"%model, sep=",", index_col=None)

In [21]:
# Print pure limits on r
MOD = 'K6'
print " alpha  Ct/CV       CV=0.5        CV=1.0        CV=1.5"
print_table([partial(read_dataframe, df=df_fits, att='median'),
             partial(read_dataframe, df=df_fits, att='downerror'),
             partial(read_dataframe, df=df_fits, att='uperror')],
             linepat=" %6.3f (%6.3f/+%5.3f)")

 alpha  Ct/CV       CV=0.5        CV=1.0        CV=1.5
-0.973 -6.000:   0.013 (-0.007/+0.007)                                              
-0.941 -4.000:   0.036 (-0.020/+0.018)                                              
-0.900 -3.000:   0.075 (-0.039/+0.036)  0.075 (-0.038/+0.036)                       
-0.862 -2.500:   0.119 (-0.061/+0.056)                                              
-0.800 -2.000:   0.202 (-0.103/+0.097)  0.202 (-0.103/+0.095)  0.202 (-0.102/+0.097)
-0.692 -1.500:   0.369 (-0.191/+0.178)  0.369 (-0.191/+0.179)                       
-0.640 -1.333:                                                 0.456 (-0.238/+0.231)
-0.610 -1.250:                          0.505 (-0.272/+0.252)                       
-0.500 -1.000:   0.685 (-0.396/+0.395)  0.686 (-0.394/+0.394)  0.685 (-0.401/+0.395)
-0.410 -0.833:                                                 0.819 (-0.526/+0.498)
-0.360 -0.750:                          0.874 (-0.601/+0.581)                       
-0.308 -0.

In [22]:
# Print fit of r * cross section * BR
def xs_fit(ct,cv, df_xsecs, df_fits, fitval='median'):
    # Take the cross section for cv=1.0, ct=ct/cv
    xsec = read_dataframe(round(ct/cv,3), 1.0, df=df_xsecs, att='tot')
    fit  = read_dataframe(ct, cv, df=df_fits, att=fitval)
    return float(xsec)*float(fit)

print " alpha  Ct/CV    CV=0.5     CV=1.0     CV=1.5"
print_table([partial(xs_fit, df_xsecs=df_xsecs[MOD], df_fits=df_fits, fitval='median'),
             partial(xs_fit, df_xsecs=df_xsecs[MOD], df_fits=df_fits, fitval='downerror'),
             partial(xs_fit, df_xsecs=df_xsecs[MOD], df_fits=df_fits, fitval='uperror')],
             linepat=" %6.3f (%6.3f/+%5.3f) ")

 alpha  Ct/CV    CV=0.5     CV=1.0     CV=1.5
-0.973 -6.000:   0.305 (-0.169/+0.155)                                                 
-0.941 -4.000:   0.322 (-0.174/+0.157)                                                 
-0.900 -3.000:   0.334 (-0.173/+0.160)   0.334 (-0.170/+0.161)                         
-0.862 -2.500:   0.341 (-0.174/+0.160)                                                 
-0.800 -2.000:   0.345 (-0.176/+0.165)   0.345 (-0.175/+0.162)   0.345 (-0.174/+0.165) 
-0.692 -1.500:   0.340 (-0.176/+0.164)   0.340 (-0.176/+0.165)                         
-0.640 -1.333:                                                   0.334 (-0.174/+0.169) 
-0.610 -1.250:                           0.328 (-0.177/+0.164)                         
-0.500 -1.000:   0.304 (-0.176/+0.175)   0.304 (-0.175/+0.175)   0.304 (-0.178/+0.175) 
-0.410 -0.833:                                                   0.276 (-0.177/+0.168) 
-0.360 -0.750:                           0.256 (-0.176/+0.170)            

#### Now do this again, but store it in a dataframe and write it to a csv file

In [23]:
df_xs_fits = pd.DataFrame()
df_xs_fits['alpha'] = np.sign(df_fits.cf*df_fits.cv)*df_fits.cf**2/(df_fits.cf**2+df_fits.cv**2)
df_xs_fits['alpha'] = df_xs_fits.alpha.round(4)
df_xs_fits['ratio'] = df_fits.cf/df_fits.cv
df_xs_fits['ratio'] = df_xs_fits.ratio.round(3)

# Use our cs_fits function to store the limits
for fitval in ['median', 'downerror', 'uperror']:
    df_xs_fits[fitval] = np.vectorize(partial(xs_fit,
                                              df_xsecs=df_xsecs[MOD],
                                              df_fits=df_fits,
                                              fitval=fitval))(df_fits.cf,
                                                              df_fits.cv)

# Remove the duplicate entries, sort, reset the index, and write to csv
df_xs_fits.drop_duplicates(subset='ratio', inplace=True)
df_xs_fits.sort_values(by='ratio', inplace=True)
df_xs_fits.index = range(1,len(df_xs_fits)+1)
df_xs_fits.to_csv("xs_fits_%s.csv"%MOD)
df_xs_fits

,alpha,ratio,median,downerror,uperror
1,-0.9730,-6.000,0.305095,-0.169196,0.155319
2,-0.9412,-4.000,0.322265,-0.173841,0.156792
3,-0.9000,-3.000,0.334266,-0.173319,0.159630
4,-0.8621,-2.500,0.340685,-0.173642,0.159721
5,-0.8000,-2.000,0.344815,-0.176303,0.165431
6,-0.6923,-1.500,0.340287,-0.176010,0.164499
7,-0.6400,-1.333,0.334446,-0.174195,0.169115
8,-0.6098,-1.250,0.328372,-0.176809,0.163707
9,-0.5000,-1.000,0.304309,-0.175619,0.175164
10,-0.4098,-0.833,0.276152,-0.177446,0.167928


#### Produce also a csv file with the pure r limits (for cv == 1)

In [26]:
df_r_fits = pd.DataFrame()
df_r_fits['alpha'] = np.sign(df_fits.cf*df_fits.cv)*df_fits.cf**2/(df_fits.cf**2+df_fits.cv**2)
df_r_fits['alpha'] = df_r_fits.alpha.round(4)
df_r_fits['ratio'] = df_fits.cf/df_fits.cv
df_r_fits['ratio'] = df_r_fits.ratio.round(3)

# Use our xs_limit function to store the limits
for fitval in ['median', 'downerror', 'uperror']:
    df_r_fits[fitval] = df_fits.get(fitval)

# Remove the duplicate entries, sort, reset the index, and write to csv
df_r_fits.drop_duplicates(subset='ratio', inplace=True)
df_r_fits.sort_values(by='ratio', inplace=True)
df_r_fits.index = range(1,len(df_r_fits)+1)
df_r_fits.to_csv("r_fits_%s.csv"%MOD)


In [27]:
df_r_fits

,alpha,ratio,median,downerror,uperror
1,-0.9730,-6.000,0.013197,-0.007318,0.006718
2,-0.9412,-4.000,0.036210,-0.019533,0.017617
3,-0.9000,-3.000,0.075469,-0.039131,0.036040
4,-0.8621,-2.500,0.119187,-0.060748,0.055877
5,-0.8000,-2.000,0.202274,-0.103422,0.097044
6,-0.6923,-1.500,0.368791,-0.190754,0.178278
7,-0.6400,-1.333,0.456252,-0.237637,0.230707
8,-0.6098,-1.250,0.504843,-0.271829,0.251686
9,-0.5000,-1.000,0.685498,-0.395606,0.394582
10,-0.4098,-0.833,0.818701,-0.526071,0.497851
